In [91]:
import numpy as np
import scipy as sp
import math

a = 2.6676e-5
n = 0.72493
n2 = 0.5

mIN = 8.1515
mAX = 93.4326
Gox = np.linspace(mIN, mAX, 200000)
#tb = 471
tb = mAX-mIN
del_t = tb/Gox.size

# Define the objective function (sum of squared residuals)
def hybrid_equation(a2):
    F1 = abs(np.sum(a*Gox**n*del_t)/tb- np.sum(a2*Gox**n2*del_t)/tb)*1000
    return F1

# Basin Hopping Methods: Powell , "bounds": bounds
result = sp.optimize.minimize(hybrid_equation,x0=8e-5)

8.5723

In [93]:
a2 = result.x
print(f"average regression rate = {(np.sum(a2*Gox**n2*del_t)/tb)*1000:.2f} mm/s")
print(a2)
print(np.sum(a*Gox**n)/Gox.size)
print(np.sum(a2*Gox**n2)/Gox.size)

average regression rate = 0.45 mm/s
[6.50786908e-05]
0.000447665453507697
0.0004476142080740326
